In [44]:
# Импорты

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [46]:
# Загрузка текста и токенизация

In [47]:
with open("gpt_data.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = ['', ''] + sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

encode = lambda s: [stoi.get(c, stoi['']) for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

encoded_data = encode(" " + text + " ")
data = torch.tensor(encoded_data, dtype=torch.long)

In [48]:
# Гиперпараметры

In [49]:
block_size = 128
batch_size = 32
embedding_dim = 128
n_heads = 4
n_layers = 2
ff_hidden = 256
n_epochs = 80
dropout_rate = 0.1
lr = 3e-4

In [50]:
# Dataset

In [51]:
class CharDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size
    def __len__(self):
        return len(self.data) - self.block_size
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

dataloader = DataLoader(CharDataset(data, block_size), batch_size=batch_size, shuffle=True)

In [52]:
# Multi-Head Attention и Transformer

In [53]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.head_dim = embed_dim // num_heads
        self.num_heads = num_heads
        self.q = nn.Linear(embed_dim, embed_dim)
        self.k = nn.Linear(embed_dim, embed_dim)
        self.v = nn.Linear(embed_dim, embed_dim)
        self.out = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, T, C = x.shape
        q = self.q(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        att = att.masked_fill(torch.tril(torch.ones(T, T, device=att.device)) == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)
        out = (att @ v).transpose(1, 2).contiguous().view(B, T, C)
        return self.out(out)

In [54]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden):
        super().__init__()
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden),
            nn.ReLU(),
            nn.Linear(ff_hidden, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = self.dropout(x)
        x = x + self.ff(self.ln2(x))
        x = self.dropout(x)
        return x

In [55]:
# MiniGPT модель

In [56]:
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, block_size, n_heads, n_layers, ff_hidden):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding = nn.Embedding(block_size, embed_dim)
        self.blocks = nn.Sequential(*[
            TransformerBlock(embed_dim, n_heads, ff_hidden)
            for _ in range(n_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        B, T = x.shape
        tok_emb = self.token_embedding(x)
        pos_emb = self.position_embedding(torch.arange(T, device=x.device))[None, :, :]
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

In [57]:
# Обучение

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniGPT(vocab_size, embedding_dim, block_size, n_heads, n_layers, ff_hidden).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        B, T, C = logits.shape
        loss = F.cross_entropy(logits.view(B * T, C), y.view(B * T))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0 or epoch == n_epochs - 1:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 3.3865
Epoch 10, Loss: 2.2122
Epoch 20, Loss: 1.8076
Epoch 30, Loss: 1.4425
Epoch 40, Loss: 1.2491
Epoch 50, Loss: 0.9015
Epoch 60, Loss: 0.6900
Epoch 70, Loss: 0.5465
Epoch 79, Loss: 0.4254


In [59]:
# Генерация текста

In [60]:
def generate(model, start_text="Привет", max_new_tokens=100, temperature=1.0, top_k=None):
    model.eval()
    context = torch.tensor(encode(start_text), dtype=torch.long)[None, :].to(device)

    for _ in range(max_new_tokens):
        context_condensed = context[:, -block_size:]
        logits = model(context_condensed)
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        if top_k is not None:
            values, indices = torch.topk(probs, top_k)
            probs = torch.zeros_like(probs).scatter_(1, indices, values)
            probs /= probs.sum(dim=-1, keepdim=True)
        next_token = torch.multinomial(probs, num_samples=1)
        context = torch.cat((context, next_token), dim=1)

    return decode(context[0].tolist())

In [61]:
# Печать результата

In [62]:
print("\nСгенерированный текст")
print(generate(model, start_text="Я бы смог завоевать весь мир одной рукой,", max_new_tokens=200, temperature=0.7, top_k=10))


Сгенерированный текст
Я бы смог завоевать весь мир одной рукой, если бы ты торую. Очни и ты бы дер. взгдала всьшай всвное всегда всела всеты всешь всешь понае пой плеанимато. это. В безнолезношемачиной льшьной бениматьносуе и ствимать ре мадатро. Сльнельноматьне.
